In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow_io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 13.7 MB/s eta 0:00:00


In [3]:
try:
  from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op  # pylint:disable=g-import-not-at-top
except ImportError:
  frontend_op = None

In [4]:
from typing import Tuple, Optional

import tensorflow as tf
from tensorflow import keras
import tensorflow_io as tfio

import numpy as np
import pandas as pd

import os
import sys
import tarfile
import hashlib
import re
import glob

import random
import math

import IPython.display as ipd
from tensorflow.python.util import compat

In [5]:

LIB_PATH = '/content/drive/MyDrive/GSC/GSC_helper'
sys.path.append(LIB_PATH)
from utils import _download, unzipzip, zipzip
from GSC import download_GSC
from GSC12 import SpeechCommands12

## GSC

In [6]:
ZIP_MAP = download_GSC('https://drive.google.com/file/d/1GDs9M1ouCw2On8Edvpy1iHdD6-sZLT9M/view?usp=drive_link',
                       'https://drive.google.com/file/d/1uExD8n1WmXWqN-nzfsbA3J-ATpQBbKy9/view?usp=drive_link',
                       'https://drive.google.com/file/d/11RpvqQn8p2ORi5W1hjUFuCQI_guz6DJd/view?usp=drive_link',
                       '/content/GSC_3',
                       end = '.zip')
CSV_MAP = download_GSC('https://drive.google.com/file/d/1-0il_aWXhlgvgwkM1ePEC4Uy1e_8Gl53/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-071VEdePvCZ5AsRoPwpQpBO9Yavzi5a/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-1v1xYN0wHSmVO5yvT5nEDErf_F3dXCd/view?usp=drive_link',
                       '/content/GSC_3',
                       end = '.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1GDs9M1ouCw2On8Edvpy1iHdD6-sZLT9M
From (redirected): https://drive.google.com/uc?id=1GDs9M1ouCw2On8Edvpy1iHdD6-sZLT9M&confirm=t&uuid=25256323-caf0-49b9-b536-9bcfd2bcf927
To: /content/GSC_3/train.zip
100%|██████████| 35.6M/35.6M [00:00<00:00, 38.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uExD8n1WmXWqN-nzfsbA3J-ATpQBbKy9
To: /content/GSC_3/val.zip
100%|██████████| 140k/140k [00:00<00:00, 5.15MB/s]
Downloading...
From: https://drive.google.com/uc?id=11RpvqQn8p2ORi5W1hjUFuCQI_guz6DJd
To: /content/GSC_3/test.zip
100%|██████████| 135k/135k [00:00<00:00, 5.14MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0il_aWXhlgvgwkM1ePEC4Uy1e_8Gl53
To: /content/GSC_3/train.csv
100%|██████████| 319k/319k [00:00<00:00, 6.34MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-071VEdePvCZ5AsRoPwpQpBO9Yavzi5a
To: /content/GSC_3/val.csv
100%|██████████| 1.26k/1.26k [00:00<00:00, 1.69MB/s]
Downloading...
From: htt

In [7]:
import pandas as pd
import os

class GSC_3(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 root: str,
                 zip_map: dict,
                 csv_map: dict,
                 unzip: bool = True,
                 subset: str = 'train',
                 batch_size: int = 32,
                 shuffle: bool = True):
        'Initialization'
        super().__init__()
        local_path = os.path.join(root, subset)
        self.root = root
        if not os.path.exists(local_path):
            os.mkdir(local_path)
            unzipzip(zip_map[subset], local_path)
        if unzip:
            unzipzip(zip_map[subset], local_path)
        self.csv = pd.read_csv(csv_map[subset])
        self.subset = subset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the numbber of batches per epoch'
        return int(np.floor(len(self.csv)/self.batch_size))

    def __getitem__(self, index):
        'Generate on batch of data'
        # Generate indexes of the batcch
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Undates indexes after each epoch'
        self.indexes = np.arange(len(self.csv))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X: (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i in indexes:
            # Store sample
            row = self.csv.iloc[i]
            X.append(np.load(os.path.join(self.root, row['link']))['arr_0'])

            # Store class
            y.append(row['label'])
        X = np.stack(X, axis = 0)
        y = np.stack(y, axis = 0)
        return tf.convert_to_tensor(X), tf.convert_to_tensor(y)

In [8]:
train_dataloader = GSC_3('/content/GSC_3', ZIP_MAP, CSV_MAP, unzip = False, subset = 'train', batch_size = 128, shuffle = True)
val_dataloader = GSC_3('/content/GSC_3', ZIP_MAP, CSV_MAP, unzip = False, subset = 'val', batch_size = 10, shuffle = False)
test_dataloader = GSC_3('/content/GSC_3', ZIP_MAP, CSV_MAP, unzip = False, subset = 'test', batch_size = 10, shuffle = False)

Extracted /content/GSC_3/train.zip
Extracted /content/GSC_3/val.zip
Extracted /content/GSC_3/test.zip


In [10]:
X, y = next(iter(train_dataloader))
X.shape

TensorShape([128, 49, 40, 1])

In [10]:
model = keras.models.Sequential([
    keras.layers.Input(shape = (49, 40, 1)),
    keras.layers.Conv2D(3, kernel_size = 5, strides = 1, padding = 'same',use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(3, kernel_size = 3, strides = 1, padding = 'same', groups = 3, use_bias = False),
    keras.layers.Conv2D(16, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(16, kernel_size = 3, strides = 1, padding = 'same', groups = 16, use_bias = False),
    keras.layers.Conv2D(32, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(32, kernel_size = 3, strides = 1, padding = 'same', groups = 32, use_bias = False),
    keras.layers.Conv2D(32, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(32, kernel_size = 3, strides = 1, padding = 'same', groups = 32, use_bias = False),
    keras.layers.Conv2D(64, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.Conv2D(64, kernel_size = 3, strides = 2, padding = 'same', groups = 64, use_bias = False),
    keras.layers.Conv2D(64, kernel_size = 1, strides = 1, use_bias = False),
    keras.layers.BatchNormalization(axis = -1),
    keras.layers.ReLU(),
    keras.layers.GlobalAveragePooling2D(keepdims = True),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(3),
    keras.layers.Softmax()])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 49, 40, 3)         75        
                                                                 
 batch_normalization_6 (Bat  (None, 49, 40, 3)         12        
 chNormalization)                                                
                                                                 
 re_lu_6 (ReLU)              (None, 49, 40, 3)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 49, 40, 3)         27        
                                                                 
 conv2d_13 (Conv2D)          (None, 49, 40, 16)        48        
                                                                 
 batch_normalization_7 (Bat  (None, 49, 40, 16)        64        
 chNormalization)                                     

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0005, weight_decay = 0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [12]:
model_ckpt = keras.callbacks.ModelCheckpoint('/content/bkup_model_ckpt',
                                             monitor = 'val_accuracy',
                                             mode = 'max',
                                             save_best_only = True,
                                             )

In [13]:
EPOCHS = 20
history = model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=EPOCHS,
    callbacks = [tf.keras.callbacks.EarlyStopping(verbose=1, patience=4),
                 model_ckpt],
)

Epoch 1/20
94/94 [==============================] - 26s 135ms/step - loss: 0.6250 - accuracy: 0.7512 - val_loss: 1.0778 - val_accuracy: 0.7000
Epoch 2/20
94/94 [==============================] - 8s 81ms/step - loss: 0.4184 - accuracy: 0.8293 - val_loss: 1.1421 - val_accuracy: 0.1500
Epoch 3/20
94/94 [==============================] - 9s 91ms/step - loss: 0.3473 - accuracy: 0.8551 - val_loss: 1.3582 - val_accuracy: 0.1500
Epoch 4/20
94/94 [==============================] - 8s 87ms/step - loss: 0.3117 - accuracy: 0.8627 - val_loss: 1.4582 - val_accuracy: 0.1500
Epoch 5/20
94/94 [==============================] - 8s 83ms/step - loss: 0.2856 - accuracy: 0.8730 - val_loss: 0.8676 - val_accuracy: 0.5167
Epoch 6/20
94/94 [==============================] - 11s 122ms/step - loss: 0.2668 - accuracy: 0.8821 - val_loss: 0.3306 - val_accuracy: 0.8167
Epoch 7/20
94/94 [==============================] - 12s 129ms/step - loss: 0.2537 - accuracy: 0.8882 - val_loss: 0.2064 - val_accuracy: 0.9167
Epoch 8

In [14]:
best_model = keras.models.load_model('/content/bkup_model_ckpt')
best_model.evaluate(test_dataloader)

5/5 [==============================] - 1s 8ms/step - loss: 0.2235 - accuracy: 0.9400


[0.2235316038131714, 0.9399999976158142]

In [15]:
_, baseline_model_accuracy = model.evaluate(test_dataloader, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9399999976158142


In [16]:
_, baseline_model_accuracy = best_model.evaluate(test_dataloader, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

Baseline test accuracy: 0.9399999976158142


## 2TFLite

In [18]:
from tqdm import tqdm

test_specs = []
test_labels = []

for i in tqdm(range(len(test_dataloader.csv))):
    row = test_dataloader.csv.iloc[i]
    spec = np.load(os.path.join('/content/GSC_3', row['link']))['arr_0']
    test_specs.append(spec)
    test_labels.append(row['label'])

test_specs = tf.convert_to_tensor(np.stack(test_specs, axis = 0))
test_labels = tf.convert_to_tensor(np.stack(test_labels))

100%|██████████| 56/56 [00:00<00:00, 980.68it/s]


In [19]:
test_specs.shape

TensorShape([56, 49, 40, 1])

In [20]:
test_labels.shape

TensorShape([56])

Baseline

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Post-train Quant Only

In [23]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(test_specs).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model_quant = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [24]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/gsc_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the dynamic range quantized model:
tflite_model_quant_file = tflite_models_dir/"gsc_tflite_backup_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

32200

In [25]:
tflite_model_file = tflite_models_dir/"gsc_model_backup.tflite"
tflite_model_file.write_bytes(tflite_model)

62524

In [26]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
    global test_specs

    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path = str(tflite_file))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    predictions = np.zeros((len(test_image_indices), ), dtype = int)
    for i, test_image_index in enumerate(test_image_indices):
        test_image = test_specs[test_image_index]

        # Check if the input type is quantized, the rescale input data to to uint8
        if input_details['dtype'] == np.int8:
            input_scale, input_zero_point = input_details['quantization']
            test_image = test_image/input_scale + input_zero_point

        test_image = np.expand_dims(test_image, axis = 0).astype(input_details['dtype'])
        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]

        predictions[i] = output.argmax()

    return predictions

# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_specs
  global test_labels

  test_image_indices = range(test_specs.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_specs)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_specs)))

In [27]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Quantized model accuracy is 94.6429% (Number of test samples=56)


In [28]:
evaluate_model(tflite_model_file, model_type="Float32")

Float32 model accuracy is 94.6429% (Number of test samples=56)


In [29]:
import shutil
shutil.copy2(str(tflite_model_quant_file), '/content')

'/content/gsc_tflite_backup_quant.tflite'

In [30]:
# Save the file as a C source file
!xxd -i /content/gsc_tflite_backup_quant_alexa.tflite > /content/bkup_model_alexa.cc
# Print the source file
!cat /content/bkup_model_alexa.cc

unsigned char _content_gsc_tflite_backup_quant_alexa_tflite[] = {
  0x20, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x00, 0x00,
  0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x1c, 0x00, 0x00, 0x00, 0x84, 0x00, 0x00, 0x00, 0xdc, 0x00, 0x00, 0x00,
  0x58, 0x3e, 0x00, 0x00, 0x68, 0x3e, 0x00, 0x00, 0x00, 0x7d, 0x00, 0x00,
  0x03, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x7e, 0xc0, 0xff, 0xff, 0x0c, 0x00, 0x00, 0x00, 0x1c, 0x00, 0x00, 0x00,
  0x3c, 0x00, 0x00, 0x00, 0x0f, 0x00, 0x00, 0x00, 0x73, 0x65, 0x72, 0x76,
  0x69, 0x6e, 0x67, 0x5f, 0x64, 0x65, 0x66, 0x61, 0x75, 0x6c, 0x74, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x98, 0xff, 0xff, 0xff,
  0x2e, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x09, 0x00, 0x00, 0x00,
  0x73, 0x6f, 0x66, 0x74, 0x6d, 0x61, 0x78, 0x5f, 0x31, 0x00, 0x00, 0x00,
  0x01, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x